In [ ]:
# Imports
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, random_split

import torchvision
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt

import copy
import os
import time

cudnn.benchmark = True

In [ ]:
# Constants
VAL_SIZE = 0.1
TEST_SIZE = 0.1
SEED = 455

In [ ]:
torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
data_dir = '/kaggle/input/rice-image-dataset/Rice_Image_Dataset'

In [ ]:
# Create datasets
image_dataset = datasets.ImageFolder(data_dir, transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]))

In [ ]:
print(image_dataset)

In [ ]:
val_length = int(len(image_dataset) * VAL_SIZE)
test_length = int(len(image_dataset) * TEST_SIZE)
train_length = len(image_dataset) - (val_length + test_length)
train_dataset, val_dataset, test_dataset = random_split(image_dataset, [train_length, val_length, test_length])

In [ ]:
image_datasets = {
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
}
dataset_sizes = { x: len(image_datasets[x]) for x in ['train', 'val', 'test'] }

In [ ]:
dataloaders = { x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=x!='test', num_workers=2)
              for x in ['train', 'val', 'test'] }

In [ ]:
class_names = image_datasets['train'].dataset.classes
print(class_names)

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from the batch
out = torchvision.utils.make_grid(inputs[:4])

# Display the images
imshow(out, title=[class_names[x] for x in classes[:4]])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        epoch_start = time.time()

        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accuracies.append(epoch_acc)
            else:
                val_losses.append(epoch_loss)
                val_accuracies.append(epoch_acc)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        epoch_time = time.time() - epoch_start
        print(f'Epoch time: {epoch_time // 60:.0f}m {epoch_time % 60:.0f}s')
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
def get_test_accuracy(model):
    correct = 0
    
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            correct += torch.sum(preds == labels.data)
    
    accuracy = correct.double() / dataset_sizes['test']
    
    print(f'Test accuracy: {accuracy:4f}')
    
    return accuracy

# SqueezeNet with SGD

In [ ]:
squeezenet1 = torchvision.models.squeezenet1_1(pretrained=True)
for param in squeezenet1.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
# Final convolution (1) of SqueezeNet has 512 input features
squeezenet1.classifier._modules['1'] = nn.Conv2d(512, 555 ,kernel_size=1)

squeezenet1 = squeezenet1.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer= optim.SGD(squeezenet1.classifier._modules['1'].parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Training with optim.SGD
squeezenet1, s1_train_loss, s1_val_loss, s1_train_acc, s1_val_acc = train_model(
    squeezenet1, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(s1_train_loss, label='Training Loss')
plt.plot(s1_val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Cross Entropy Loss for SqueezeNet with SGD')
plt.savefig('SqueezeNet SGD Loss.png')
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot([acc.cpu() for acc in s1_train_acc], label='Training Accuracy')
plt.plot([acc.cpu() for acc in s1_val_acc], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy for SqueezeNet with SGD')
plt.savefig('SqueezeNet SGD Accuracy.png')
plt.show()

In [ ]:
torch.save(squeezenet1.state_dict(), 'squeezenet1.pth')

In [ ]:
visualize_model(squeezenet1)

plt.show()

In [ ]:
s1_test_accuracy = get_test_accuracy(squeezenet1)

# SqueezeNet with Adadelta

In [ ]:
squeezenet2 = torchvision.models.squeezenet1_1(pretrained=True)
for param in squeezenet2.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
# Final convolution (1) of SqueezeNet has 512 input features
squeezenet2.classifier._modules['1'] = nn.Conv2d(512, 555 ,kernel_size=1)

squeezenet2 = squeezenet2.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer= optim.SGD(squeezenet1.classifier._modules['1'].parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Training with optim.SGD
squeezenet2, s2_train_loss, s2_val_loss, s2_train_acc, s2_val_acc = train_model(
    squeezenet2, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(s2_train_loss, label='Training Loss')
plt.plot(s2_val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Cross Entropy Loss for SqueezeNet with Adadelta')
plt.savefig('SqueezeNet Adadelta Loss.png')
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot([acc.cpu() for acc in s2_train_acc], label='Training Accuracy')
plt.plot([acc.cpu() for acc in s2_val_acc], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy for SqueezeNet with Adadelta')
plt.savefig('SqueezeNet Adadelta Accuracy.png')
plt.show()

In [ ]:
torch.save(squeezenet1.state_dict(), 'squeezenet2.pth')

In [ ]:
visualize_model(squeezenet2)

plt.show()

In [ ]:
s2_test_accuracy = get_test_accuracy(squeezenet2)

# SqueezeNet with RMSprop

In [ ]:
squeezenet3 = torchvision.models.squeezenet1_1(pretrained=True)
for param in squeezenet3.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
# Final convolution (1) of SqueezeNet has 512 input features
squeezenet3.classifier._modules['1'] = nn.Conv2d(512, 555 ,kernel_size=1)

squeezenet3 = squeezenet3.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer= optim.SGD(squeezenet3.classifier._modules['1'].parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Training with optim.SGD
squeezenet3, s3_train_loss, s3_val_loss, s3_train_acc, s3_val_acc = train_model(
    squeezenet3, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(s3_train_loss, label='Training Loss')
plt.plot(s3_val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Cross Entropy Loss for SqueezeNet with RMSprop')
plt.savefig('SqueezeNet RMSprop Loss.png')
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot([acc.cpu() for acc in s3_train_acc], label='Training Accuracy')
plt.plot([acc.cpu() for acc in s3_val_acc], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy for SquezeNet with RMSprop')
plt.savefig('SquezeNet RMSprop Accuracy.png')
plt.show()

In [ ]:
torch.save(squeezenet1.state_dict(), 'squeezenet3.pth')

In [ ]:
visualize_model(squeezenet3)

plt.show()

In [ ]:
s3_test_accuracy = get_test_accuracy(squeezenet3)